# Basic Cashflow Computation
## Compute Constant Cashflow
This method will provide a constant cashflow of *'Initial Value'*.

The overall cash flow length is determined by the *'length'* parameter, while the initial number of periods, specified by the *'shift'* parameter, is zero.

In [0]:
public static double[] GetConstantValues(this double initialValue, int shift, int length){

    var constant = new double[length];
    
    for (var i = shift; i < length; i++)
        constant[i] =  initialValue;
        
    return constant;
}

## Compute Linear Cashflow
This method provides a linearly increasing cash flow.

The amount of which is equal to the "**Initial Value**" plus any "**Values**" for the given time period.

The overall cash flow length is determined by the *'length'* parameter, while the initial number of periods, specified by the *'shift'* parameter, is zero.

In [0]:
public static double[] GetLinearValues(this double initialValue, int shift, int length, double[] values){

    var linear = new double[length];
    
    for (var i = shift; i < length; i++)
        linear[i] =  initialValue + values.Take(i - shift + 1).Aggregate((x, y) => x + y);
        
    return linear;
}

## Compute Compounded Cashflow
This method provides a geometrically increasing or decreasing cash flow.

The value of the cash flow in the first period is equal to "**Initial Value**" * (1 + **Value**(t)). In subsequent periods, the cash flow is equal to the previous cash flow multiplied by (1 + **Value**(t)).

The overall cash flow length is determined by the *'length'* parameter, while the initial number of periods, specified by the *'shift'* parameter, is zero.

In [0]:
public static double[] GetCompoundValues(this double initialValue, int shift, int length, double[] values){
    
    var compound = new double[length];

    for (var i = shift; i < length; i++)
        compound[i] = initialValue * values.Take(i - shift + 1).Select(x => 1 + x).Aggregate((x, y) => x * y);
             
    return compound;
}

# Bounded Cashflow Computation

## Compute Bounded Random Cashflows

This method provides random cash flow amounts, with values between the specified "**Minimum**" and "**Maximum**" boundaries.

The overall cash flow length is determined by the *'length'* parameter, while the initial number of periods, specified by the *'shift'* parameter, is zero.

In [0]:
public static double[] GetBoundedRandomValues(this int shift, int length, double max, double min){
    var bounded = new double[length];
    var random = new Random();

    for (var i = shift; i < length; i++)
        bounded[i] = random.NextDouble() * (max - min) + min;
               
    return bounded;
}

# Correlate Risk Cashflows

Correlating Cash Flows for Risk Drivers in the Solvency Framework.

Some cash flows for risk drivers in the Solvency framework can be correlated. This is done by using a correlation factor.

Source: [Eiopa CEIOPS Correlation Parameters](https://register.eiopa.europa.eu/CEIOPS-Archive/Documents/Advices/CEIOPS-L2-Advice-Correlation-Parameters.pdf)

## Multiple Risks

The correlated value of multiple risks is calculated as follows:

$$
\text{Correlated Value} = \sum_{i,j} \text{r}_i \text{r}_j \text{Corr}(\text{r}_i,\text{r}_j)
$$

- $$\text{r}_i$$ and $$\text{r}_j$$ are the risk measures of the individual risks

The correlation effect is calculated as follows:

$$
\text{Correlation Effect} = \text{Correlated Value} - \sum_i \text{r}_i
$$

In [0]:
public static double GetCorrelatedValue(double[] r1, double[] r2, double[,] correlation){

   return Math.Sqrt(r1.Select((x,i) => r2.Select((y,j) => x * y * correlation[i,j] ).Sum()).Sum());

}

## Two risks (not implemented)
The correlated value of two correlated risks is calculated as follows:

$$
\text{Correlated Value} = \sqrt{ r_1^2 + r_2^2 + 2 \text{r}_1 \text{r}_2  \text{Corr}(\text{r}_1,\text{r}_2)  }
$$
where:

- $$r_1$$ and $$r_2$$ are the risk measures of the two correlated risks
- $$\text{Corr}(\text{r}_1,\text{r}_2)$$ is the correlation factor between the two risk measures
- $$r_1$$ and $$r_2$$ are the weights of the two risk measures

The correlation effect is the difference between the correlated value and the sum of the two risk measures:

$$
\text{Correlation Effect} = \text{Correlated Value} - \text{r}_1 - \text{r}_2
$$

If both risks are the same, the correlation is assumed to be 1, and the correlation effect is 0. Therefore, there is no need to provide correlation factors of 1.